In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import gridspec
import scipy.ndimage as ndimage
import timeit

In [2]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
# pandas for nice tables
import pandas as pd

In [6]:
mcc = MouseConnectivityCache(manifest_file='mcc/manifest.json', resolution=25)

In [7]:
av, _ = mcc.get_annotation_volume()
av.shape

(528L, 320L, 456L)

In [8]:
tv, _ = mcc.get_template_volume()
tv.shape

(528L, 320L, 456L)

## Function for rotating a point around a plane

In [58]:
def rotatePoint(template, angle, point, plane):
    new_point = np.array([point[0],point[1],point[2]])

    degrees = angle
    x = point[plane[0]]
    y = point[plane[1]]
    L = template.shape[plane[0]]
    H = template.shape[plane[1]]
    
    newL = (L * np.sin(np.radians(degrees)) + H * np.cos(np.radians(degrees)))
    newH = (L * np.cos(np.radians(degrees)) + H * np.sin(np.radians(degrees)))

    new_point[plane[1]] = ((x - 0.5 * L) * np.cos(np.radians(degrees))) - ((y - 0.5 * H) * np.sin(np.radians(degrees))) + 0.5 * newH
    new_point[plane[0]] = ((x - 0.5 * L) * np.sin(np.radians(degrees))) + ((y - 0.5 * H) * np.cos(np.radians(degrees))) + 0.5 * newL
    return new_point

## Function for rotating the voxels given an angle

In [59]:
def rotate1d(template, angle, plane):
    rotated = ndimage.interpolation.rotate(template, angle, reshape=True, axes=plane)
    return rotated;

## Function that does the above two given the starting template, angles in the order of [sagittal, horizontal, coronal] and also a point if you want

In [60]:
def rotate3d(template, angles, point=-1):
    sagittal = (0, 1)
    horizontal = (0, 2)
    coronal = (2, 1)
    
    rotated_1 = rotate1d(template, angles[0], sagittal)
    rotated_2 = rotate1d(rotated_1, angles[1], horizontal)
    rotated_3 = rotate1d(rotated_2, angles[2], coronal)

    if point == -1:
        return rotated_3;
    else:
        point_1 = rotatePoint(template, angles[0], point, sagittal)
        point_2 = rotatePoint(rotated_1, angles[1], point_1, horizontal)
        point_3 = rotatePoint(rotated_2, angles[2], point_2, coronal)
        
        return rotated_3, point_3;

## Example Usage

In [64]:
start_time = timeit.default_timer()

rotation_angles = [45, 45, 45]
rotation_point = [280, 160, 228]

tv_rotated, new_point = rotate3d(template = tv, angles = rotation_angles, point = rotation_point)

elapsed = timeit.default_timer() - start_time
print str(elapsed) + ' seconds'

NameError: name 'elapsed' is not defined

In [67]:
print tv_rotated[new_point[0], new_point[1], new_point[2]]
print new_point

141
[381 473 489]
